Team: HiddenLayer

Fullnames: Γιάνναρος Χρήστος, Δραμιτινός Μηχάλης και Μπίνας Βαλαβάνης Βασίλης.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try :
  # In Google Colab
  %tensorflow_version 2.x
except :
  pass

# Import Tensorflow
import tensorflow as tf

# Assure TensorFlow version >= 2 is installed. Otherwise install TensorFlow 2.0 version.
try :
  assert tf.__version__ >= "2.0"
  print ("TensorFlow version found:", tf.__version__)
except :
  !pip install tensorflow==2.4

TensorFlow version found: 2.4.1


In [ ]:
!pip install Music21

# Import needed modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, io, requests
from music21 import converter, instrument, note, chord

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/clean_midi/clean_midi" #path to clean_midi folder that is inside clean_midi.

In [ ]:
path1 ="/content/drive/MyDrive/midi" #path to midi.

In [ ]:
artists = os.listdir(path) #a list of every artist 
songs = []
number_of_songs_per_artist = []     # a loop that gives as a list with every song 
for i in range(len(artists)):
  s = 0
  if os.path.isdir(os.path.join(path, artists[i])):
    for j in  os.listdir(os.path.join(path, artists[i])):
      s+=1
      songs.append(j)
    number_of_songs_per_artist.append(s)

In [ ]:
songs1 = []     #a loop that gives us the name of the songs of the midi folder
for j in  os.listdir(os.path.join(path1)):
  songs1.append(j)

In [ ]:
len(songs1) #number of songs

In [ ]:
songs1.sort() #sorting the names of the songs alphabetically

In [ ]:
for name in songs1: #names of all the songs
  print(name)

In [ ]:
notes_per_artist = []                 #the loop below gives us an array with every note from all the songs of each artist 
counter = 0                           
counter_2 = 0
counter_3 = 0
number_of_artists = artists[:15]

for artist in number_of_artists:
  os.chdir(os.path.join(path, artist))
  counter_3 = counter_2
  notes_1 = []
  for i in range(number_of_songs_per_artist[counter]):
    counter_2 += 1
    if counter_2 in [21,79,256]:  #these are the potitions of the songs that can't be opened because of errors
      pass
    else: 
      midi = converter.parse(songs[counter_3 + i])
      notes_1_to_parse = None
      parts = instrument.partitionByInstrument(midi)
      if parts: # file has instrument parts
        notes_1_to_parse = parts.parts[0].recurse()
      else: # file has notes in a flat structure
          notes_1_to_parse = midi.flat.notes
      for element in notes_1_to_parse:
          if isinstance(element, note.Note):
              notes_1.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes_1.append('.'.join(str(n) for n in element.normalOrder))
  notes_per_artist.append(notes_1)
  counter += 1

In [ ]:
#the loop bellow gives us an array with every note from all the songs of each artist 

os.chdir(os.path.join(path1))
notes = []
for i in range(len(songs1)):
  notes1 = []
  midi = converter.parse(songs1[i])
  notes_to_parse = None
  parts = instrument.partitionByInstrument(midi)
  if parts: # file has instrument parts
      notes_to_parse = parts.parts[0].recurse()
  else: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes
  for element in notes_to_parse:
      if isinstance(element, note.Note):
          notes1.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
          notes1.append('.'.join(str(n) for n in element.normalOrder))
  notes.append(notes1)

Bellow we have the names of the songs used from clean_midi with their artists:

Local H:

Back in the Day.mid
Bound for the Floor.mid
Bag of Hammers.mid
Bound for the Floor.1.mid

Little Texas:

God Bless Texas.mid

Loesser:

Never Will I Marry.mid
Don't Cry.mid
The Most Happy Fella.mid

Loft:

Love Is Magic.mid

In [ ]:
notes_artist = notes_per_artist[0]
N = 4 #number of artists that we will take notes from

for i in range(1,N):
  notes_artist = np.append(notes_artist, notes_per_artist[i])  #all the notes from the songs of the first N artists 

#Now we will create another dataset for our transfer model
notes_artist1 = notes[0]

N = 19 #number of artists that we will take notes from

for i in range(1,N):
  if i not in [10,12]:
    notes_artist1 = np.append(notes_artist1, notes[i])  #all the notes from the all the songs except 2

notes_artist2 = np.append(notes[10], notes[12]) 

# get all pitch names
pitchnames = sorted(set(item for item in notes_artist))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

#same thing for the dataset of the transfer model
# get all pitch names
pitchnames1 = sorted(set(item for item in notes_artist1))
# create a dictionary to map pitches to integers
note_to_int1 = dict((note, number) for number, note in enumerate(pitchnames1))

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back = 1) :
  dataX, dataY = [], []
  for i in range( len(dataset) - look_back ) :
    a = dataset[i : i + look_back]
    dataX.append(a)
    dataY.append(dataset[i + look_back])
  return np.array(dataX), np.array(dataY)

In [ ]:
window = 90

# Training set :
sequence_in = notes_artist[:]
train = [note_to_int[char] for char in sequence_in]

sequence_in = notes_artist1[:]
train1 = [note_to_int1[char] for char in sequence_in]

notes_for_the_final_product = [note_to_int1[char] for char in notes_artist2[:]]

In [ ]:
train_x , train_y = create_dataset(train, window)
train_x1 , train_y1 = create_dataset(train1, window)

from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot vector.
train_y_c = to_categorical(train_y)
train_y_c1 = to_categorical(train_y1)
number_of_output = train_y_c.shape[1]
number_of_output1 = train_y_c1.shape[1]

In [ ]:
# Convert to LSTM friendly format 
x_train_lstm = np.reshape(train_x, (-1, window, 1))
x_train_lstm1 = np.reshape(train_x1, (-1, window, 1))

In [ ]:
np.random.seed(45)
tf.random.set_seed(45)

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout

# Create a model
model = Sequential()
model.add(LSTM(256, input_shape = (window , 1)))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Dense(126))
model.add(Dense(number_of_output))
model.add(Activation('softmax'))

model.summary()

model.compile(loss = "categorical_crossentropy", optimizer = "adam")

history = model.fit(x_train_lstm, train_y_c,
                    epochs = 50, batch_size = 32)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 126)               32382     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               25400     
_________________________________________________________________
activation (Activation)      (None, 200)               0

In [ ]:
# Create a Sequential model.
model1 = Sequential()

# Load the extracted features from model3 (remove the classification layers).
for layer in model.layers[:-6]:
  layer.trainable = False # DO NOT train these layers.
  model1.add(layer)

# Initialize to random weights the classification layers and train them.
model1.add(Dense(256))
model1.add(Dropout(0.2))
model1.add(Dense(128))
model1.add(Dense(128))
model1.add(Dropout(0.25))
model1.add(Dense(64))
model1.add(Dense(number_of_output1, activation = 'softmax'))

# Print the model.
model1.summary()

# Compile the model.
model1.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Fit the model.
history1 = model1.fit(x_train_lstm1, train_y_c1,
                    epochs = 100, batch_size = 32)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dense_12 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_14 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)               

In [ ]:
import random as rn

#starting with a random array of 90 notes

start = rn.randint(0, len(train1)-window)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames1))
pattern = train1[ start : start + window]
prediction_output = []

# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (-1, window, 1))
    prediction = model1.predict(prediction_input)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

In [ ]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.4 

In [ ]:
from music21 import stream
#creating the music file
final_product = stream.Stream(output_notes)
final_product.write('midi', fp='/content/drive/MyDrive/music_train.mid')

'/content/drive/MyDrive/music_train.mid'